---
# LEAFToolbox - SL2P 

This notebook contains code blocks to run SL2P based on three different inputs, which are as follows:
1. using SL2p 20m net and 10m net for processing NEON simulated data-output 10m result
2. using SL2p 20m net for processing NEON simulated data-input 20m, output 20m result1
3. using SL2p 20m net and 10m net for processing DenS2 data-input 10m, output 10m result
4. using SL2p 20m net and 10m net for processing S2, 20m net output -20m, 10m net output -10m
5. ALR estimate (input 10m band, output 10m, no nueral network net used in this procedure) 
---

In [2]:
import ee
import time
import math
import csv
import json
import os
import numpy as np
import pandas as pd
import folium ; from folium import plugins
import matplotlib.pyplot as plt
import scipy ; from scipy import stats
import scipy.io as sio
import sklearn as skl ; from sklearn import linear_model ; from sklearn import preprocessing
import tensorflow as tf
import pickle
from collections import OrderedDict

# import custom modules (files must be in same directory as this notebook)
import feature_collections as fc
import image_bands as ib
import wrapper_nets as wn
import ee_functions as ee_func
import ALR_functions as alr

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWtgzh61eZqJ6LMW-oqI3JmKNuAy5O92hds5dVLsBsRRzL6iqrABnr7IsvY



Successfully saved authorization token.


---
# Prelim: Define dictionaries

In [230]:
# -----------------------
# SELECT INPUT PARAMETERS
# -----------------------

# variable name
# one of: 'fAPAR', 'fCOVER', 'LAI'
#outputName = 'LAI'
outputName = 'LAI'

# site selection
# one of: 'Geraldton', 'FoxCreek', 'Kouchibouguac', 'Ottawa',
#         'Wabush', 'QueenCharlotte', 'Attawapiskat', 'Eastmain', 'Charlottetown', 'RedBay', 'EaglePlain', 'Kitchener'
#siteSelect = 'Charlottetown'
siteSelect = 'ABBY'
#ABBY, HOPB, SERC, STEI, UNDE, MCDI,LENO, NOGP,JORN, SJER

# assetfolder='users/GangHong2/NEON'

assetfolder_REF  = 'users/ccrs12fy2022simha/DOWNSCALING_PROCESS/REF'
assetfolder_NULL  ='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/NULL'

assetfolder_SOL_A  ='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A'
assetfolder_SOL_B  ='users/ccrs12fy2022simhaDOWNSCALING_PROCESS//SOL_B'
assetfolder_SOL_C  ='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_C'

assetfolder_RES_D  ='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/RES_D'
assetfolder_RES_E  ='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/RES_E'
assetfolder_RES_F  ='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/RES_F'

In [231]:
# -----------------------------------------------------
# set parameters based on user-defined parameters above
# -----------------------------------------------------
outputParams = {
    'fAPAR': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'fCOVER': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 1
    },
    'LAI': {
        'outputScale': 1000,
        'outputOffset': 0,
        'outputMax': 8
    }
}

outputScale = outputParams[outputName]['outputScale']
outputOffset = outputParams[outputName]['outputOffset']
outputMax = outputParams[outputName]['outputMax']
responseBand = 'estimate'+outputName

In [232]:
siteParams = {
    # # Geraldton, ON
    # 'Geraldton': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200811T164849_20200811T165525_T16UEA'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-86, 49.5], \
    #                       [-86, 50], \
    #                       [-85.5, 50], \
    #                       [-85.5, 49.5]]]),
    #     'mapCenter': [-85.75, 49.75]
    # },
    # # Fox Creek, AB
    # 'FoxCreek': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20210825T185919_20210825T190431_T11UNA'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-117, 54], \
    #                       [-117, 55], \
    #                       [-115, 55], \
    #                       [-115, 54]]]),
    #     'mapCenter': [-116.8, 54.4]
    # },
    # # Kouchibouguac, NB
    # 'Kouchibouguac': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200905T151701_20200905T151829_T20TLS'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-65, 46], \
    #                       [-65, 47], \
    #                       [-64, 47], \
    #                       [-64, 46]]]),
    #     'mapCenter': [-64.5, 46.5]
    # },
    # # Ottawa, ON
    # 'Ottawa': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200801T155911_20200801T160644_T18TVQ'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-75, 45], \
    #                       [-75, 46], \
    #                       [-74, 46], \
    #                       [-74, 45]]]),
    #     'mapCenter': [-74.5, 45.5]
    # },
    # # Wabush, NL
    # 'Wabush': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200815T153911_20200815T154107_T19UFU'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-67.5, 52.3], \
    #                       [-67.5, 53.2], \
    #                       [-66.3, 53.2], \
    #                       [-66.3, 52.3]]]),
    #     'mapCenter': [-67, 52.8]
    # },
    # # Queen Charlotte Island, BC
    # 'QueenCharlotte': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200909T194951_20200909T195633_T08UPE'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-133, 53.2], \
    #                       [-133, 54], \
    #                       [-132, 54], \
    #                       [-132, 53.2]]]),
    #     'mapCenter': [-132.4, 53.6]
    # },
    # # Attawapiskat, ON
    # 'Attawapiskat': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200815T162839_20200815T163731_T17ULU'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-83, 52.3], \
    #                       [-83, 53.2], \
    #                       [-82.4, 53.2], \
    #                       [-82.4, 52.3]]]),
    #     'mapCenter': [-82.7, 52.7]
    # },
    # # Eastmain, QC
    # 'Eastmain': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200723T161829_20200723T162656_T17UPT'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-79.5, 51.4], \
    #                       [-79.5, 52.3], \
    #                       [-78, 52.3], \
    #                       [-78, 51.4]]]),
    #     'mapCenter': [-78.7, 51.8]
    # },
    # # Charlottetown, PEI
    # 'Charlottetown': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200622T151659_20200622T151653_T20TMS'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-63.3, 46.1], \
    #                       [-63.3, 46.5], \
    #                       [-62.9, 46.5], \
    #                       [-62.9, 46.1]]]),
    #     'mapCenter': [-63.1, 46.3]
    # },
    # # Red Bay, NL
    # 'RedBay': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200716T145729_20200716T145730_T21UWT'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-56.6, 51.6], \
    #                       [-56.6, 52.3], \
    #                       [-55.6, 52.3], \
    #                       [-56.6, 51.6]]]),
    #     'mapCenter': [-56, 52]
    # },
    # # Eagle Plain, YT
    # 'EaglePlain': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200731T204019_20200731T204021_T08WMU'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-137, 65.75], \
    #                       [-137, 66.5], \
    #                       [-135, 66.5], \
    #                       [-135, 65.75]]]),
    #     'mapCenter': [-136.3, 66.5]
    # },
    # # Kitchener, ON
    # 'Kitchener': {
    #     'testImage': ee.Image('COPERNICUS/S2_SR/20200615T160911_20200615T161838_T17TNJ'),
    #     'mapBounds': ee.Geometry.Polygon( \
    #                     [[[-81, 43.3], \
    #                       [-81, 44], \
    #                       [-79.8, 44], \
    #                       [-79.8, 43.3]]]),
    #     'mapCenter': [-80.5, 43.7]
    # },
    # ABBY
    'ABBY': {
        'testImage': ee.Image("users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
     },
    'HOPB': {
        'testImage': ee.Image("users/GangHong2/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'SERC': {
        'testImage': ee.Image("users/GangHong2/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'STEI': {
        'testImage': ee.Image("users/GangHong2/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'UNDE': {
        'testImage': ee.Image("users/GangHong2/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'MCDI': {
        'testImage': ee.Image("users/GangHong2/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'LENO': {
        'testImage': ee.Image("users/GangHong2/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },  \
    'NOGP': {
        'testImage': ee.Image("users/GangHong2/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D09_NOGP_DP1_20200626_152700_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'JORN': {
        'testImage': ee.Image("users/GangHong2/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'WREF': {
        'testImage': ee.Image("users/GangHong2/NEON_D16_WREF_DP1_20190712_173947_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D16_WREF_DP1_20190712_173947_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    },
    'SJER': {
        'testImage': ee.Image("users/GangHong2/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m"),
        'mapBounds': ee.Image("users/GangHong2/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m").geometry()
        # 'mapCenter': [-80.5, 43.7]
    }
}
# all other sites, just follow the site 'ABBY' to add new sites
# users/GangHong2/NEON_D01_HOPB_DP1_20190826_172857_reflectance_10m
# users/GangHong2/NEON_D02_SERC_DP1_20210811_142655_reflectance_10m
# users/GangHong2/NEON_D05_STEI_DP1_20190608_194643_reflectance_10m
# users/GangHong2/NEON_D05_UNDE_DP1_20190606_184411_reflectance_10m
# users/GangHong2/NEON_D06_MCDI_DP1_20200713_192937_reflectance_10m
# users/GangHong2/NEON_D08_LENO_DP1_20210422_172136_reflectance_10m
# users/GangHong2/NEON_D09_NOGP_DP1_20190722_170025_reflectance_10m
# users/GangHong2/NEON_D14_JORN_DP1_20190825_165611_reflectance_10m
# users/GangHong2/NEON_D16_WREF_DP1_20190712_173947_reflectance_10m
# users/GangHong2/NEON_D17_SJER_DP1_20210331_200812_reflectance_10m


mapBounds = siteParams[siteSelect]['mapBounds']
# mapCenter = siteParams[siteSelect]['mapCenter']
testImage = siteParams[siteSelect]['testImage']

# other filters
# maxCloudcover = 10

# export parameters
exportFolder = siteSelect+'_'+outputName
exportDataType = 'int'
exportScale = 20

In [233]:
COLLECTION_OPTIONS = {
    # Sentinel 2 using 20 m bands:
    'COPERNICUS/S2_SR': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'COPERNICUS/S2_SR_10m': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      "Watercover": 'WATER_PERCENTAGE',
      "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
    # Sentinel 2 using 10 m bands:
    'NEON_Sim_10m': {
      "name": 'NEON_Sim_10m',
      "description": 'NEON Simulated',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_10m_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
     'NEON_Sim': {
      "name": 'COPERNICUS/S2_SR',
      "description": 'Sentinel 2A',
      # "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
      # "Watercover": 'WATER_PERCENTAGE',
      # "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
      # "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
      # "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE', 
      # "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
      "VIS_OPTIONS": 'VIS_OPTIONS',
      "Collection_SL2P": ee.FeatureCollection(fc.s2_createFeatureCollection_estimates()),
      "Collection_SL2Perrors": ee.FeatureCollection(fc.s2_createFeatureCollection_errors()),  
      "sl2pDomain": ee.FeatureCollection(fc.s2_createFeatureCollection_domains()),
      "Network_Ind": ee.FeatureCollection(fc.s2_createFeatureCollection_Network_Ind()),
      "partition": ee.ImageCollection(fc.s2_createImageCollection_partition()),
      "legend": ee.FeatureCollection(fc.s2_createFeatureCollection_legend()),
      "numVariables": 7
    },
}

VIS_OPTIONS = {
    'fAPAR': {
        "COPERNICUS/S2_SR": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim_10m": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
         "NEON_Sim": {
            "Name": 'fAPAR',
            "errorName": 'errorfAPAR',
            "maskName": 'maskfAPAR',
            "description": 'Fraction of absorbed photosynthetically active radiation',
            "variable": 2,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
    },
    'fCOVER': {
        "COPERNICUS/S2_SR": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim_10m": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        },
         "NEON_Sim": {
            "Name": 'fCOVER',
            "errorName": 'errorfCOVER',
            "maskName": 'maskfCOVER',
            "description": 'Fraction of canopy cover',
            "variable": 3,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]]))) 
        }
    },
    'LAI': {
        "COPERNICUS/S2_SR": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "COPERNICUS/S2_SR_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
          "NEON_Sim_10m": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B2', 'B3', 'B4', 'B8'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        },
        "NEON_Sim": {
            "Name": 'LAI',
            "errorName": 'errorLAI',
            "maskName": 'maskLAI',
            "description": 'Leaf area index',
            "variable": 1,
            "inputBands":      ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
            "inputScaling":    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "outmin": (ee.Image(ee.Array([[0]]))),
            "outmax": (ee.Image(ee.Array([[1]])))
        }
    }
}

In [234]:
### for NEON bands
def  renameBands(image):
    bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7','b8','b9','b10','b11','b12','b13','b14','b15','b16']
    new_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'cosVZA','cosSZA','cosRAA']
    return image.select(bands).rename(new_bands)

---
# 1 - SL2P/SL2P10 - Processing NEON Simulated - Independent step

## SL2P (input bands 10m, output band 10m, use 20m neural network coefficents) -Reference

In [ ]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [ ]:
# filter collection and add ancillary bands

# input_collection = ee.ImageCollection(testImage) \
#                      .map(lambda image: ib.addDate(image)) \
#                      .map(lambda image: image.clip(mapBounds)) \
#                      .map(lambda image: ib.s2MaskClear(image)) \
#                      .map(lambda image: ib.s2MaskLand(image)) \
#                      .map(lambda image: ib.addS2Geometry(colOptions, image))
input_collection = ee.ImageCollection(testImage).map(renameBands)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name +"_"+siteSelect +"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name[16:]+"_"+outputName+"_"+"20m_net" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[16:], 16 here means the lengh of users/GangHong2/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [ ]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,
                                               # image_names=[siteSelect+'_'+outputName+'_SL2P'],
                                               image_names = image_output_names,
                                               scale=10,
                                               # asset_folder='users/kateharvey/SL2P_images',
                                               asset_folder=assetfolder_REF,
                                               data_type='float',
                                               max_pixels=1e13)

## SL2P10 (input bands 10m, output 10m,  10m neural network coefficients) -solution B

In [ ]:
# parse the networks
colName ='NEON_Sim_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [ ]:
input_collection_10m = ee.ImageCollection(testImage).map(renameBands)

In [ ]:
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

In [ ]:
# print (scaled_input_collection_10m.first().getInfo())

In [ ]:
# performs same procedure as above, using SL2P10 network
# applies algorithm to 10 m bands ; generates a 10 m map

# filter collection and add ancillary bands
# input_collection_10m = ee.ImageCollection(testImage) \
#                      .map(lambda image: ib.addDate(image)) \
#                      .map(lambda image: image.clip(mapBounds)) \
#                      .map(lambda image: ib.s2MaskClear(image)) \
#                      .map(lambda image: ib.s2MaskLand(image)) \
#                      .map(lambda image: ib.addS2Geometry(colOptions, image))
# input_collection_10m = ee.ImageCollection(testImage)

# get partition used to select network
# partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')
# partition = (COLLECTION_OPTIONS[colName]["partition"]).rename('partition')

# # pre process input imagery and flag invalid inputs
# scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
#                                                   .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))




# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name[16:]+"_"+outputName for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()])

ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [ ]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,
                                                   # image_names=[siteSelect+'_'+outputName+'_SL2P10'],
                                                   image_names = image_output_names_10m,
                                                   scale=10,
                                                   # asset_folder='users/kateharvey/SL2P10_images',
                                                   asset_folder=assetfolder_SOL_B,
                                                   # data_type=exportDataType,
                                                   data_type='float',
                                                   max_pixels=1e13)

# 2 - SL2P - Processing 20m NEON - Independent step

## SETUP: resample 10m NEON to 20m

In [191]:
inputcollection_10m = ee.ImageCollection(testImage).map(renameBands) ## rename testImage bands based on the bands name for input SL2P
proj_10m=inputcollection_10m.first().select('B1').projection().getInfo()  ## get the projection of one band
inputImage_20m = inputcollection_10m.first().resample('bilinear').reproject(crs=proj_10m['crs'], scale=20) ## resample the image to 20m

## SL2P (input band 20m, ouput band 20m, 20m net) -NULL Hypothesis

In [192]:
# parse the networks
colName = 'NEON_Sim'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [193]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(inputImage_20m)##.map(renameBands)
# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name +"_"+siteSelect +"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name[16:61]+"_20m_"+outputName+"_"+"20m_net" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).get('system:id')).getInfo()]) ##[name[16:], 16 here means the lengh of users/GangHong2/ 

ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [194]:
# export tasks to Earth Engine
export_task_20m = ee_func.export_collection_to_gee(collection=export_collection,
                                                   num_images=1,                                                   
                                                   image_names = image_output_names,
                                                   scale=10,   ## inputband 20m, the acutal output is 20m, but here 10m for comparison with others                                            
                                                   asset_folder=assetfolder_NULL,                                                   
                                                   data_type='float',
                                                   max_pixels=1e13)

# 3 - SL2P/SL2P10- Processing DenS2 - Independent step

## SETUP 1

In [40]:
### make an imagecollection from DenS2 imageries- 10 totally
img1=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20190822T173909_N0213_R098_T13SCS_20190822T220238'))
img2=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20190826T153819_N0213_R011_T18TYN_20190826T195901'))
img3=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210422T162829_N0300_R083_T16SCA_20210422T204151'))
img4=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'))
img5=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20190608T164849_N0212_R026_T15TYL_20190608T212115'))
img6=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210401T183919_N0300_R070_T11SKB_20210401T224235'))
img7=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20200627T173909_N0214_R098_T14TLS_20200627T213600'))
img8=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210808T154809_N0301_R054_T18SUJ_20210808T201351'))
img9=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2A_MSIL2A_20200713T170901_N0214_R112_T14SQJ_20200713T213310'))
img10=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2A_MSIL2A_20190606T165901_N0212_R069_T16TCS_20190606T212006'))

## an image collection
DenS2_Col=img1.merge(img2).merge(img3).merge(img4).merge(img5).merge(img6).merge(img7).merge(img8).merge(img9).merge(img10)


## SETUP 2: preparing image for runing SL2P

In [41]:
### get the asset ID, like 'users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
neon_id=testImage.get("system:id").getInfo()
neon_name=neon_id[16:]   ## slice the neon_id to get the name, like 'NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
## based on the property name 'NEON' to find denS2
denS2=DenS2_Col.filter(ee.Filter.eq('NEON', neon_name))
## get the DenS2 name, like 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'
denS2_Name=denS2.first().get("system:id").getInfo()[22:]
## find S2 from GEE from denS2 file name
S2img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(denS2.geometry()).filter(ee.Filter.eq('PRODUCT_ID', denS2_Name))
### select S2 based on DenS2 image geometry and keep band 'SCL' for masking Land
S2_selected=S2img.first().clip(denS2.geometry()).select('SCL')
## merge S2 SCL band and DenS2 bands
newS2=S2_selected.addBands(denS2.first())
## rename bands 
inputImage_bands = ee.List(['SCL','B4', 'B3', 'B2', 'B8','B5', 'B6', 'B7', 'B8A', 'B11', 'B12'])
inputImage = newS2.rename(inputImage_bands)
# print(inputImage.bandNames().getInfo())

## SL2P (input band 10m, output band 10m, 20m net) -Solution A

In [25]:
# parse the networks
colName = 'COPERNICUS/S2_SR'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [26]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(inputImage).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name+"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name+"_"+siteSelect+"_"+outputName+"_DenS2_20m_net" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [27]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                             
                                               image_names = image_output_names,
                                               scale=10,                                          
                                               asset_folder=assetfolder_SOL_A,
                                               data_type='float',
                                               max_pixels=1e13)

## SL2P10 (inputband 10m, output band 10m, 10m net)

In [28]:
# parse the networks
colName = 'COPERNICUS/S2_SR_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [29]:
# performs same procedure as above, using SL2P10 network
# applies algorithm to 10 m bands ; generates a 10 m map

# filter collection and add ancillary bands
input_collection_10m = ee.ImageCollection(inputImage).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.s2MaskLand(image)) \
                                                  .map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_DenS2_10m_net" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [30]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,                                                  
                                                   image_names = image_output_names_10m,
                                                   scale=10,                                                  
                                                   asset_folder=assetfolder_RES_D,                                                
                                                   data_type='float',
                                                   max_pixels=1e13)

# 4 - SL2P/SL2P10 - Processing S2 - Independent step

## SETUP

In [195]:
### make an imagecollection from DenS2 imageries
img1=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20190822T173909_N0213_R098_T13SCS_20190822T220238'))
img2=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20190826T153819_N0213_R011_T18TYN_20190826T195901'))
img3=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210422T162829_N0300_R083_T16SCA_20210422T204151'))
img4=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'))
img5=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20190608T164849_N0212_R026_T15TYL_20190608T212115'))
img6=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210401T183919_N0300_R070_T11SKB_20210401T224235'))
img7=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20200627T173909_N0214_R098_T14TLS_20200627T213600'))
img8=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2B_MSIL2A_20210808T154809_N0301_R054_T18SUJ_20210808T201351'))
img9=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2A_MSIL2A_20200713T170901_N0214_R112_T14SQJ_20200713T213310'))
img10=ee.ImageCollection(ee.Image('users/GangHong2/DenS2/S2A_MSIL2A_20190606T165901_N0212_R069_T16TCS_20190606T212006'))

## an image collection
DenS2_Col=img1.merge(img2).merge(img3).merge(img4).merge(img5).merge(img6).merge(img7).merge(img8).merge(img9).merge(img10)


In [196]:
### get the asset ID, like 'users/GangHong2/NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
neon_id=testImage.get("system:id").getInfo()
neon_name=neon_id[16:]   ## slice the neon_id to get the name, like 'NEON_D16_ABBY_DP1_20210719_191207_reflectance_10m'
## based on the property name 'NEON' to find denS2
denS2=DenS2_Col.filter(ee.Filter.eq('NEON', neon_name))
## get the DenS2 name, like 'S2B_MSIL2A_20210719T190919_N0301_R056_T10TER_20210719T215339'
denS2_Name=denS2.first().get("system:id").getInfo()[22:]
## find S2 from GEE from denS2 file name
S2img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(denS2.geometry()).filter(ee.Filter.eq('PRODUCT_ID', denS2_Name))
### select S2 based on DenS2 image geometry and keep band 'SCL' for masking Land
S2_selected=S2img.first().clip(denS2.geometry()).select(['B2', 'B3', 'B4', 'B5','B6', 'B7', 'B8', 'B8A', 'B11', 'B12','SCL'])

## SL2P (input bands 20m, output 20m,  20m net)

In [197]:
# parse the networks
colName = 'COPERNICUS/S2_SR'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [198]:
# filter collection and add ancillary bands
input_collection = ee.ImageCollection(S2_selected).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection = input_collection.map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                          .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P = scaled_input_collection.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P = estimateSL2P.map(lambda image: image.addBands(image.select("estimate"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(image.select("error"+outputName).multiply(ee.Image.constant(outputScale)).add(ee.Image.constant(outputOffset)), overwrite=True))

# produce final export collection
export_collection = input_collection.combine(estimateSL2P).combine(uncertaintySL2P)
    
# image_output_names = ([name+"_"+outputName for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names = ([name+"_"+siteSelect+"_"+outputName+"_S2_20m_net" for name in export_collection.toList(export_collection.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [199]:
# export tasks to Earth Engine
export_task = ee_func.export_collection_to_gee(collection=export_collection,
                                               num_images=1,                                             
                                               image_names = image_output_names,
                                               scale=20,                ## output 20m                
                                               asset_folder=assetfolder_RES_E,
                                               data_type='float',
                                               max_pixels=1e13)

## SL2P10 (input bands 10m, output 10m, 10m net)

In [200]:
# parse the networks
colName = 'COPERNICUS/S2_SR_10m'
colOptions = COLLECTION_OPTIONS[colName]
netOptions = VIS_OPTIONS[outputName][colName]
numNets = ee.Number(ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove('system:index').remove('lon').size())
SL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,netNum));
errorsSL2P = ee.List.sequence(1,ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]),1).map(lambda netNum: wn.makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"],numNets,netNum));

In [201]:
# performs same procedure as above, using SL2P10 network
# applies algorithm to 10 m bands ; generates a 10 m map

# filter collection and add ancillary bands
input_collection_10m = ee.ImageCollection(S2_selected).map(lambda image: ib.s2MaskLand(image)).map(lambda image: ib.addS2Geometry(colOptions, image))

# get partition used to select network
partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition')

# pre process input imagery and flag invalid inputs
scaled_input_collection_10m = input_collection_10m.map(lambda image: ib.s2MaskLand(image)) \
                                                  .map(lambda image: ib.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],image)) \
                                                  .map(lambda image: ib.invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"],netOptions["inputBands"],image))

# apply networks to produce mapped parameters
estimateSL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image, outputName))
uncertaintySL2P_10m = scaled_input_collection_10m.map(lambda image: wn.wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image, outputName))

# scale and offset mapped parameter bands
estimateSL2P_10m = estimateSL2P_10m.map(lambda image: image.addBands(image.select("estimate"+outputName) \
                                                             .multiply(ee.Image.constant(outputScale)) \
                                                             .add(ee.Image.constant(outputOffset)), overwrite=True))
uncertaintySL2P_10m = uncertaintySL2P_10m.map(lambda image: image.addBands(image.select("error"+outputName) \
                                                                   .multiply(ee.Image.constant(outputScale)) \
                                                                   .add(ee.Image.constant(outputOffset)), overwrite=True))


# produce final export collection
export_collection_10m = input_collection_10m.combine(estimateSL2P_10m).combine(uncertaintySL2P_10m)

# image_output_names_10m = ([name+"_"+outputName+"_10m" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
image_output_names_10m = ([name+"_"+siteSelect+"_"+outputName+"_S2_10m_net" for name in export_collection_10m.toList(export_collection_10m.size()).map(lambda image: ee.Image(image).id()).getInfo()])
ee_func.displayImage(export_collection_10m.mosaic().select('estimate'+outputName),0+outputOffset,10*outputScale+outputOffset, mapBounds)

In [202]:
# export tasks to Earth Engine
export_task_10m = ee_func.export_collection_to_gee(collection=export_collection_10m,
                                                   num_images=1,                                                  
                                                   image_names = image_output_names_10m,
                                                   scale=10,                                                  
                                                   asset_folder=assetfolder_RES_F,                                                
                                                   data_type='float',
                                                   max_pixels=1e13)

# 5. ALR estimate (input 10m band, output 10m, no nueral network net but random forest used in this procedure)-independent step Solution C

In [235]:
## ALR function for estimation,after feature selection, a simple random forest estiamte applied
def func_ALR(temp_image, responsedBand, outputName, mapBounds):
   
    # inputImage = ee.Image(temp_image).select(1,2,3,7,22,23,27,28,29,30,31,32)
    # inputImage = ee.Image(temp_image).select(1,2,3,7,16,17,18,19,20,21)
    # inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'QA60', 'date', 'estimate'+outputName, 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    inputImage_bands = ee.List(['B2', 'B3', 'B4', 'B8', 'estimate'+outputName, 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    # inputImage = inputImage.rename(inputImage_bands)
    inputImage = ee.Image(temp_image).select(inputImage_bands)
    input_VI_definition = ee.List([
                                   "GI      = b('B3')/b('B4')",                               
                                   "SGI     = b('B8')/b('B4')",                                
                                   "GVI     = (b('B8')/b('B3'))-1",                             
                                   "NDVI3   = ((b('B8')-b('B4'))/(b('B8')))+b('B4')",                                
                                   "NDVI    = (b('B8')-b('B4'))/(b('B8')+b('B4'))",
                                   "GNDVI   = (b('B8')-b('B3'))/(b('B8')+b('B3'))",                                
                                   "NDGI    = (b('B3')-b('B4'))/(b('B3')+b('B4'))",                                 
                                   "EVI     = 2.5*((b('B8')-b('B4'))/(b('B8')+6*b('B4')-7.5*b('B3')+1))",
                                   "EVI2    = 2.5*((b('B8')-b('B4'))/(b('B8')+2.4*b('B4')+1))",
                                   "RDVI    = (b('B8')-b('B4'))/((b('B8')+b('B4'))**0.5)",
                                   "MSR     = ((b('B8')/b('B4'))-1)/((b('B8')/b('B4'))**0.5+1)",                            
                                   "MSAVI2  = 0.5*(2*b('B8')+1-((2*b('B8')+1)**2-8*(b('B8')-b('B4')))**0.5)",                                
                                   "NLI     = ((b('B8')**2)-b('B4'))/((b('B8')**2)+b('B4'))"])

    # names of bands to pass to ALR method (excluding metadata and other non-spectral bands)
    input_bandNames = ['B2', 'B3', 'B4', 'B8', 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI', 'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI']
    
    def format_image(image, image_bands, response_band, VI_definition):
        image = ee.Image(image)
        image_bands = ee.List(image_bands)
        response_band = ee.String(response_band)
        VI_definition = ee.List(VI_definition)
        
        # image_bands specifices a list of the names of the bands used in defining the expressions for VIs in VI_definition
        image = image.rename(image_bands).toDouble()
        
        # Generate an ImageCollection from a list of expressions defining a set of VIs using the bands available in the image
        VIimageCollection = ee.ImageCollection(VI_definition.map(lambda expr: image.expression(expr)))
        VIimage = VIimageCollection.toBands().regexpRename("[0-9]+_", "")
        
        # Reorder the bands in the image so the response band is the first band in the image
        feature_bands = image_bands.remove(response_band)
        
        return ee.Image(image.select(response_band).addBands(VIimage).addBands(image.select(feature_bands)))

    
    inputImage = format_image(inputImage, inputImage_bands, responseBand, input_VI_definition)
    
    def scale_image(image, response_band):
        image = ee.Image(image)
        response_band = ee.String(response_band)
        
        def get_num_pixels(image):
    
            # get image height
            def get_height(image):
                height = image.getInfo()["bands"][0]["dimensions"][0]
                return height

            # get image width
            def get_width(image):
                width = image.getInfo()["bands"][0]["dimensions"][1]
                return width

            image_height = get_height(image)
            image_width = get_width(image)
            image_pixels = image_height*image_width

            return image_pixels
        
        image_pixels = ee.Number(get_num_pixels(image))
        
        # Set up lists containing the input/feature bands in the image
        bandList = image.bandNames()
        featureList = bandList.remove(response_band)
        num_bands = bandList.length()
        num_features = featureList.length()
        
        # We will be using the reduceRegion() function on images from Earth Engine, 
        # which will process up to a specified number of pixels from the image to generate the outputs of the reducer
        max_pixels = image_pixels.min(10000000)
        # best_effort = ee.Algorithms.If(image_pixels.gt(max_pixels), True, False)
        
        # Set default projection and scale using the response band
        defaultScale = image.select(response_band).projection().nominalScale()
        defaultCrs = image.select(response_band).projection().crs()
        image = image.setDefaultProjection(crs=defaultCrs, scale=defaultScale)
        
        # Center all of the bands in the image for LARs
        # We will centre the sampled data later as well as reduceRegion() is not precise enough
        meanImage = image.subtract(image.reduceRegion(reducer=ee.Reducer.mean(), \
                                    scale=defaultScale, bestEffort=True, maxPixels=max_pixels).toImage(bandList))
        
        # Separate the image into features (X) and response (y) as we need to standardize the input features
        X = meanImage.select(featureList)
        y = meanImage.select(response_band)
        
        # Standardize the input features
        X = X.divide(X.reduceRegion(reducer=ee.Reducer.stdDev(), bestEffort=True, maxPixels=max_pixels).toImage(featureList))
        
        return X.addBands(y)

    
    scaledImage = scale_image(inputImage, responseBand)
    
    
    def ee_LARS(input_image, input_bandNames, response_bandName, num_nonzero_coefficients, num_samples):
        image = ee.Image(input_image)
        feature_list = ee.List(input_bandNames)
        response_band = ee.String(response_bandName)
        full_band_list = ee.List(feature_list).add(response_band)
        num_nonzero_coefficients = ee.Number(num_nonzero_coefficients)
        num_samples = ee.Number(num_samples)
        def get_num_pixels(image):
    
            # get image height
            def get_height(image):
                height = image.getInfo()["bands"][0]["dimensions"][0]
                return height

            # get image width
            def get_width(image):
                width = image.getInfo()["bands"][0]["dimensions"][1]
                return width

            image_height = get_height(image)
            image_width = get_width(image)
            image_pixels = image_height*image_width

            return image_pixels
        image_pixels = ee.Number(get_num_pixels(image))
        
        # Randomly sample pixels in the image at native resolution into a FeatureCollection
        input_collection = image.sample(numPixels=num_samples.min(image_pixels))
        n = input_collection.size()
        m = feature_list.length()
        
        # Use an aggregate array function over the FeatureCollection and map the function over each feature in the band list
        # to generate a dictionary of all of the samples retrieved
        inputs = ee.Dictionary.fromLists(full_band_list, full_band_list.map(lambda feature: input_collection.aggregate_array(feature)))
        
        # Although we may call our scale_image function on the input image, the reduceRegion() function used to determine the mean
        # and standard deviation of each band in the image over the entire region is not precise enough over a large image
        # so we must recenter all of the bands in the image and now we can also normalize (L2 norm) each input feature as required
        # by the LARs algorithm
        
        # Use an aggregate_mean function over the feature collection to get the mean of each band
        input_means = ee.Dictionary.fromLists(full_band_list, full_band_list.map(lambda feature: input_collection.aggregate_mean(feature)))

        def centre_inputs(key, value):
            key_mean = input_means.getNumber(key)
            return ee.List(value).map(lambda sample: ee.Number(sample).subtract(key_mean))
        
        
        # Center bands by mapping over the list of features and then a subtracting over the list of samples for each band
        inputs = inputs.map(centre_inputs)

        # Separate the response variable samples into its own vector
        y = inputs.toArray([response_band]).reshape([-1,1])

        # Remove response band from the feature collection by selecting only bands in the feature list
        inputs = inputs.select(feature_list)
        
        # Generate a dictionary of all of the L2 norms of the input features using a custom mapped function
        input_norms = inputs.map(lambda key, value: ee.Number(ee.List(value).map(lambda sample: ee.Number(sample).pow(2)).reduce(ee.Reducer.sum())).pow(0.5))

        def norm_inputs(key, value):
            key_norm = input_norms.getNumber(key)
            return ee.List(value).map(lambda sample: ee.Number(sample).divide(key_norm))
        
        # Normalize all of the features by mapping a function over the list of features
        # and then map a division over the list of all of the samples of the feature
        inputs = inputs.map(norm_inputs)
        
        # Generate the array of samples using the dictionary
        X = inputs.toArray(feature_list).transpose()

        # Find the first best predictor of the response to initialize the main LARs loop
        initial_prediction = ee.Array(ee.List.repeat([0], n))
        c = X.transpose().matrixMultiply(y.subtract(initial_prediction))
        c_abs = c.abs()
        C_maxLoc = c_abs.project([0]).argmax()
        add_feature = C_maxLoc.getNumber(0)
        A = ee.List([add_feature])
        
        # Create a dicitionary of initial inputs to pass into the main LARs iterative loop
        # The iterate function in Earth Engine processes each iteration as a tree of iterations with no access to any variables
        # from previous iterations (only those that are passed to the next iteration)
        # so we must pass both the current prediction and the active set of features (with non-zero coefficients), A
        initial_inputs = ee.Dictionary({'prediction': initial_prediction, 'A': A})

        def LARs_regression(iteration, inputs):
            inputs = ee.Dictionary(inputs)

            # Find the active set of features, A (predictors with non-zero coefficients)
            A = ee.List(inputs.get('A'))
            # A_list is an array used to mask the full array of input samples and the correlation vector
            A_list = ee.Array(ee.List.sequence(0, m.subtract(1))\
                              .map(lambda index: A.contains(index)).replaceAll(False, 0).replaceAll(True, 1)).reshape([-1,1])

            # The following matrix algebra determines the next most correlated variable, or the next best predictor considering the
            # current features in the active set, A, as well as the magnitude to adjust the prediction vector to ensure all of the
            # features in the active set are equally correlated to response vector
            prediction = inputs.getArray('prediction')
            c = X.transpose().matrixMultiply(y.subtract(prediction))
            c_abs = c.abs()
            C_max = c_abs.get(c_abs.argmax())
            s_A = c.divide(c_abs).mask(A_list)
            X_A = X.mask(A_list.transpose())
            G_Ai = X_A.transpose().matrixMultiply(X_A).matrixInverse()
            G1 = G_Ai.matrixMultiply(s_A)
            A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
            w_A = G1.multiply(A_A)
            u_A = X_A.matrixMultiply(w_A)
            a = X.transpose().matrixMultiply(u_A)
            a = a.project([0])
            c = c.project([0])

            def compute_gammaArray(index_j):
                minus_j = C_max.subtract(c.get([index_j])).divide(A_A.subtract(a.get([index_j])))
                plus_j = C_max.add(c.get([index_j])).divide(A_A.add(a.get([index_j])))
                return ee.List([minus_j, plus_j]).filter(ee.Filter.gte('item', 0)).reduce(ee.Reducer.min())

            A_c = ee.List.sequence(0, m.subtract(1)).removeAll(A)
            gammaArray = A_c.map(compute_gammaArray)
            gamma = gammaArray.reduce(ee.Reducer.min())
            min_location = gammaArray.indexOf(gamma)
            add_feature = A_c.getNumber(min_location)

            # Update active set of variables with next best predictor from non-active set and update prediction vector
            A = A.add(add_feature)
            prediction = prediction.add(u_A.multiply(gamma))

            return ee.Dictionary({'prediction': prediction, 'A': A})


        # The final iteration of LARs (if selecting all input variables) requires a different method to determine magnitude for
        # adjusting the magnitude of the prediction vector, as the regular LARs iteration relies on variables in non-active set
        # In the final iteration there will be no variables in the non-active set, so the method will not work
        def LARs_final_iteration(iteration, inputs):
            inputs = ee.Dictionary(inputs)
            A = ee.List(inputs.get('A'))

            prediction = inputs.getArray('prediction')
            c = X.transpose().matrixMultiply(y.subtract(prediction))
            c_abs = c.abs()
            C_max = c_abs.get(c_abs.argmax())        

            s_A = c.divide(c_abs)
            G_Ai = X.transpose().matrixMultiply(X).matrixInverse()
            G1 = G_Ai.matrixMultiply(s_A)
            A_A = s_A.project([0]).dotProduct(G1.project([0])).pow(-0.5)
            w_A = G1.multiply(A_A)
            u_A = X.matrixMultiply(w_A)

            gamma = C_max.divide(A_A)
            prediction = prediction.add(u_A.multiply(gamma))

            return ee.Dictionary({'prediction': prediction, 'A': A})

        # Actually carrying out the iterations by iterating over a placeholder list (sequence from 1 to the number of non-zero
        # variables that the user wishes to select as predictors for the response)
        iterations = ee.List.sequence(1, m.subtract(1).min(num_nonzero_coefficients))
        penultimate_outputs = iterations.iterate(LARs_regression, initial_inputs)
        final_outputs = ee.Dictionary(ee.Algorithms.If(num_nonzero_coefficients.gte(m), \
                                LARs_final_iteration(m, penultimate_outputs), penultimate_outputs))
        
        final_prediction = final_outputs.getArray('prediction')

        A = ee.List(final_outputs.get('A'))

        feature_path = A.slice(0, num_nonzero_coefficients).map(lambda index: feature_list.getString(index))        
        return feature_path

    
    select_features = ee_LARS(scaledImage, input_bandNames, responseBand, 5, 50000)
    #unclassified = ee.Image(cloud_folder+'/'+siteSelect+'_'+outputName+'_VI')
    unclassified = ee.Image(inputImage)
    # bands = ee.List([responseBand, 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI',
    #                  'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI', 'B2', 'B3', 'B4', 'B8',
    #                  'QA60', 'date', 'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    bands = ee.List([responseBand, 'GI', 'SGI', 'GVI', 'NDVI3', 'NDVI', 'GNDVI', 'NDGI',
                     'EVI', 'EVI2', 'RDVI', 'MSR', 'MSAVI2', 'NLI', 'B2', 'B3', 'B4', 'B8',
                     'partition', 'networkID', 'error'+outputName, 'partition_1', 'networkID_1'])
    unclassified = unclassified.rename(bands)

    # prediction bands (equivalent to select_features, with responseBand)
    bands = select_features
    input_bands = select_features.add(responseBand)
    training_data = ee.FeatureCollection(unclassified.sample(numPixels=1000, seed=1).select(input_bands))
    # implement regression tree with Random Forest algorithm
    # optional parameters for smileRandomForest(): variablesPerSplit, minLeafPopulation, bagFraction, maxNodes, seed
    rf_classifier = ee.Classifier.smileRandomForest(100).setOutputMode('REGRESSION').train(features=training_data,
                                                                                           classProperty=responseBand,
                                                                                           inputProperties=input_bands)
    rf_classified = unclassified.select(bands).classify(rf_classifier, 'ALR_'+responseBand).clip(mapBounds)
    return temp_image.addBands(rf_classified)
    

In [239]:
## use a biophysical parameter result as an input

assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20210719T190919_20210719T191700_T10TER_ABBY_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20190826T153819_20190826T154455_T18TYN_HOPB_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20210808T154809_20210808T155521_T18SUJ_SERC_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20190608T164849_20190608T165019_T15TYL_STEI_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20190606T165901_20190606T170333_T16TCS_UNDE_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20200713T170901_20200713T171937_T14SQJ_MCDI_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20210422T162829_20210422T163638_T16SCA_LENO_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20200627T173909_20200627T174744_T14TLS_NOGP_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20190822T173909_20190822T175453_T13SCS_JORN_LAI_DenS2_20m_net'
# assetname='users/ccrs12fy2022simha/DOWNSCALING_PROCESS/SOL_A/20210401T183919_20210401T184709_T11SKB_SJER_LAI_DenS2_20m_net'

#ABBY, HOPB, SERC, STEI, UNDE, MCDI,LENO, NOGP,JORN, SJER

# assetname='users/GangHong2/NEON/20210719T190919_20210719T191700_T10TER_ABBY_LAI_DenS2_20m_net'
img=ee.Image(assetname)
mapBounds=img.geometry()
output_Name=assetname[21:]+'_ALR' ### index number neeed to be adjusted to get the meaningful file name '20210719T190919_20210719T191700_T10TER_ABBY_LAI_DenS2_20m_net_ALR'20210401T183919_20210401T184709_T11SKB_SJER_LAI_DenS2_20m_net

In [240]:
outputName = 'LAI'
responseBand = 'estimate'+outputName
ALR_result=func_ALR(img, responseBand, outputName, mapBounds)
# print (ALR_result.bandNames().getInfo())

In [241]:
# export formatted image to GEE asset 
assetfolder='users/GangHong2/ALR'
export = ee.ImageCollection(ALR_result)
export_task = ee_func.export_collection_to_gee(collection=export,
                                                 num_images=1,                                                                                              
                                                 # image_names=[siteSelect+'_'+outputName+'_ALR'],
                                                 image_names=[output_Name],
                                                 asset_folder=assetfolder_SOL_C,                                                 
                                                 scale=10,
                                                 data_type='float',
                                                 max_pixels=1e13)

EEException: Invalid description. The description must contain only the following characters: a..z, A..Z, 0..9, ".", ",", ":", ";", "_" or "-". The description must be at most 100 characters long.